In [1]:
import pandas as pd
import os
import base64
import time
import pickle as p
import numpy as np
import matplotlib.pyplot as plt
import pandas.core.algorithms as algos
from sklearn.model_selection import train_test_split
from sklearn import metrics
from functools import reduce
import logging
import matplotlib.style as style
import matplotlib
import matplotlib.lines as mlines
from matplotlib.ticker import FuncFormatter
style.use('seaborn')

import warnings
warnings.filterwarnings('ignore')

In [2]:
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

In [3]:
import gc
gc.collect()

51

### Data Loading and Preprocessing

In [4]:
Client = pd.read_excel('../data/Client_info_20191118.xlsx',sheet_name='Sheet1')
TPA = pd.read_excel('../data/Copy_of_TPA_cash.xlsx',sheet_name='Sheet1')
Client = Client.merge(TPA,how='left',left_on='Client ID',right_on='HOST ID')
Customer = pd.read_csv('../data/Customer_Data_Request_20191121.csv')

In [5]:
Client_cols = ['Active',
 'Client ID',
 'Account Name',
 'Industry',
 'NAICS Code',
 'NAICS Description',
 'Broker',
 'TPA_x',
 'Launch Date',
 'Termination Date',
 'Affiliate/Fed Gov?',
 'Cash']

In [6]:
Customer = Customer.merge(Client[Client_cols],how='left',left_on='Unique_Company_ID',right_on='Client ID')

In [7]:
Perf = pd.read_csv('../data/Performance_Data_Request_20191204.csv')
Perf = Perf.merge(Customer, how='left',on='Unique_Customer_ID')
Perf['Year_and_Month'] = pd.to_datetime(Perf['Year_and_Month'].astype(str), format = '%Y%m')
# Perf['YQ'] = pd.PeriodIndex(Perf['Year_and_Month'], freq='Q')
# Perf.to_csv('./data/Perf.csv',index = False)

In [8]:
Perf['YQ'] = pd.PeriodIndex(Perf['Order_Date'], freq='Q')

TypeError: Incorrect dtype

In [ ]:
Perf.loc[Perf['Order_Amount']=='.','Order_Amount']=0
Perf['Order_Amount'] = Perf['Order_Amount'].astype(float)

In [ ]:
# Assign Payment Order as a new column
Perf['Payment_Order'] = Perf.groupby('Unique_Order_ID')['Year_and_Month'].rank(method='first')

In [13]:
Perf.shape

(38397651, 51)

In [9]:
Perf.head()

,Unique_Order_ID,Unique_Customer_ID,Order_Amount,Product_Markup,Payments_Due,Current_Balance,Date_account_went_delinquent,Default_Status,Delinquency_Bucket,Losses,Auto_Pay_Enrollment,Order_Closed_Flag,Type_of_Backup_Payment,Spending_Limit_at_time_of_Order,Tenure_at_time_of_Order,Salary_at_time_of_Order,Street1_x,Street2_x,City_x,State_x,Zip_x,items_ordered,Year_and_Month,EMPSTATUS,Order_Date,Unique_Company_ID,Enrollment_Date,Assigned_Spending_Limit,Customer_Tenure,Customer_Salary,Prior_Customer_Flag,Street1_y,Street2_y,City_y,State_y,Zip_y,Status,Term_Date,Active,Client ID,Account Name,Industry,NAICS Code,NAICS Description,Broker,TPA_x,Launch Date,Termination Date,Affiliate/Fed Gov?,Cash,YQ
0,10000091,100486182,1072.88,.,40.11,907.86,.,Current,000-000,0.0,PA,N,ACH,96.6,327,800.29,929 IMPERIAL CT,NaN,HALETHORPE,MD,21227.0,DeWalt 12in 15-Amp Double-Bevel Compound Miter...,2014-04-01,Employed,03/15/2014,2049.0,02/25/2014,9660.0,327.0,80029.0,Repeat Buyer,929 IMPERIAL CT,NaN,HALETHORPE,MD,21227,Employed,NaN,Y,2049.0,FEDERAL GOVERNMENT/POSTAL WORKERS/NIH/DEPT OF ...,Federal Government/Associations,NaN,NaN,NaN,NaN,2002-07-01,NaT,Fed Gov,NaN,2014Q2
1,10000091,100486182,1072.88,.,40.11,817.86,.,Current,000-000,0.0,PA,N,ACH,96.6,327,800.29,929 IMPERIAL CT,NaN,HALETHORPE,MD,21227.0,DeWalt 12in 15-Amp Double-Bevel Compound Miter...,2014-05-01,Employed,03/15/2014,2049.0,02/25/2014,9660.0,327.0,80029.0,Repeat Buyer,929 IMPERIAL CT,NaN,HALETHORPE,MD,21227,Employed,NaN,Y,2049.0,FEDERAL GOVERNMENT/POSTAL WORKERS/NIH/DEPT OF ...,Federal Government/Associations,NaN,NaN,NaN,NaN,2002-07-01,NaT,Fed Gov,NaN,2014Q2
2,10000091,100486182,1072.88,.,40.11,727.86,.,Current,000-000,0.0,PA,N,ACH,96.6,327,800.29,929 IMPERIAL CT,NaN,HALETHORPE,MD,21227.0,DeWalt 12in 15-Amp Double-Bevel Compound Miter...,2014-06-01,Employed,03/15/2014,2049.0,02/25/2014,9660.0,327.0,80029.0,Repeat Buyer,929 IMPERIAL CT,NaN,HALETHORPE,MD,21227,Employed,NaN,Y,2049.0,FEDERAL GOVERNMENT/POSTAL WORKERS/NIH/DEPT OF ...,Federal Government/Associations,NaN,NaN,NaN,NaN,2002-07-01,NaT,Fed Gov,NaN,2014Q2
3,10000091,100486182,1072.88,.,40.11,637.86,.,Current,000-000,0.0,PA,N,ACH,96.6,327,800.29,929 IMPERIAL CT,NaN,HALETHORPE,MD,21227.0,DeWalt 12in 15-Amp Double-Bevel Compound Miter...,2014-07-01,Employed,03/15/2014,2049.0,02/25/2014,9660.0,327.0,80029.0,Repeat Buyer,929 IMPERIAL CT,NaN,HALETHORPE,MD,21227,Employed,NaN,Y,2049.0,FEDERAL GOVERNMENT/POSTAL WORKERS/NIH/DEPT OF ...,Federal Government/Associations,NaN,NaN,NaN,NaN,2002-07-01,NaT,Fed Gov,NaN,2014Q3
4,10000091,100486182,1072.88,.,40.11,547.86,.,Current,000-000,0.0,PA,N,ACH,96.6,327,800.29,929 IMPERIAL CT,NaN,HALETHORPE,MD,21227.0,DeWalt 12in 15-Amp Double-Bevel Compound Miter...,2014-08-01,Employed,03/15/2014,2049.0,02/25/2014,9660.0,327.0,80029.0,Repeat Buyer,929 IMPERIAL CT,NaN,HALETHORPE,MD,21227,Employed,NaN,Y,2049.0,FEDERAL GOVERNMENT/POSTAL WORKERS/NIH/DEPT OF ...,Federal Government/Associations,NaN,NaN,NaN,NaN,2002-07-01,NaT,Fed Gov,NaN,2014Q3


### Create C/O Flag by criteria --- 150 DPD/No Pay last 90 

In [12]:
Perf['Balance_diff_past_90'] = Perf['Current_Balance'].diff(-3)
# Filter where the difference is calculated between different orders, and then assign NaN to the value
mask = Perf['Unique_Order_ID']!= Perf['Unique_Order_ID'].shift(3)
Perf['Balance_diff_past_90'][mask]=np.nan

In [10]:
# Perf[['Unique_Order_ID','Current_Balance','Balance_diff_past_90']].head(50)

In [11]:
# def CO_Flag(row):
#     if (row['Delinquency_Bucket']=='151+') and (row['Balance_diff_past_90']==0):
#         return 1
#     else:
#         return 0

In [9]:
Perf_af15 = Perf.groupby('Unique_Order_ID').filter(lambda x: x['Year_and_Month'].min()>=pd.to_datetime('2015-01'))

In [10]:
Perf_af15.shape

(28922383, 51)

In [15]:
Perf['CO_Flag'] = 0
Perf.loc[(Perf['Delinquency_Bucket']=='151+') & (Perf['Balance_diff_past_90']==0),'CO_Flag']=1

In [18]:
# def new_DQ(row):
#     if row['CO_Flag'] ==1 :
#         return 'CO'
#     else:
#         return row['Delinquency_Bucket']

In [19]:
# Perf['CO_Flag'] = Perf.apply(CO_Flag,axis=1)
# Perf['DQ_new']  = Perf.apply(new_DQ,axis = 1)

In [16]:
Perf['DQ_new'] = Perf['Delinquency_Bucket']
Perf.loc[Perf['CO_Flag']==1,'DQ_new'] = 'CO'

In [17]:
Perf.groupby(['Default_Status','CO_Flag']).size()

Default_Status  CO_Flag
001-030         0           2977970
031-060         0            823220
061-090         0            586676
091-120         0            483602
121-150         0            427299
151+            0           1115044
                1            326051
Bankruptcy      0            165796
                1             41306
Chargeoff       0            351118
                1               682
Current         0          30940967
Fraud           0            156978
                1               942
dtype: int64

In [20]:
Perf.groupby('YQ').agg({'CO_Flag':'sum','Unique_Order_ID':'count'}).reset_index()

,YQ,CO_Flag,Unique_Order_ID
0,2012Q1,0,50256
1,2012Q2,0,143959
2,2012Q3,118,256584
3,2012Q4,1543,431733
4,2013Q1,1994,605728
5,2013Q2,2100,655469
6,2013Q3,2808,694354
7,2013Q4,3618,764010
8,2014Q1,4794,788795
9,2014Q2,5735,788875


In [19]:
CO_Loss = Perf[Perf['Default_Status']=='Chargeoff'].groupby(['YQ']).agg({'Losses':'sum'}).reset_index()
Total_Balance = Perf.groupby(['YQ']).agg({'Current_Balance':'sum'}).reset_index()
CO_Loss.merge(Total_Balance, how='left', on= 'YQ')

,YQ,Losses,Current_Balance
0,2012Q2,970.06,1.536808e+08
1,2012Q3,191187.73,2.389975e+08
2,2012Q4,493415.99,3.617630e+08
3,2013Q1,536864.12,4.423111e+08
4,2013Q2,2428065.53,4.123733e+08
5,2013Q3,2611896.68,3.902052e+08
6,2013Q4,3971190.71,4.451606e+08
7,2014Q1,3655656.25,4.967926e+08
8,2014Q2,3509867.46,4.476229e+08
9,2014Q3,3396924.30,4.320898e+08


In [30]:
Perf.pivot_table(values = 'Current_Balance',index = 'YQ', columns = 'DQ_new',aggfunc='sum',margins = True)

DQ_new,000-000,001-030,031-060,061-090,091-120,121-150,151+,CO,All
YQ,,,,,,,,,
2012Q1,5.895969e+07,1.946948e+06,4.995665e+04,2.330500e+03,NaN,NaN,NaN,NaN,6.095892e+07
2012Q2,1.380004e+08,1.320173e+07,1.693510e+06,5.819009e+05,1.815513e+05,1.943262e+04,2.330500e+03,NaN,1.536808e+08
2012Q3,2.067315e+08,2.340078e+07,3.890096e+06,2.214250e+06,1.383073e+06,9.144608e+05,3.145098e+05,1.487480e+05,2.389975e+08
2012Q4,3.178057e+08,2.699850e+07,6.203654e+06,3.670709e+06,2.539918e+06,1.892901e+06,9.918885e+05,1.659741e+06,3.617630e+08
2013Q1,3.752310e+08,4.156449e+07,7.586047e+06,4.271571e+06,3.593581e+06,2.964612e+06,5.181238e+06,1.918543e+06,4.423111e+08
2013Q2,3.377268e+08,4.131305e+07,1.055375e+07,6.246150e+06,4.692835e+06,3.593183e+06,6.446113e+06,1.801403e+06,4.123733e+08
2013Q3,3.132123e+08,3.764253e+07,9.942882e+06,6.907057e+06,5.613185e+06,4.818267e+06,1.011873e+07,1.950193e+06,3.902052e+08
2013Q4,3.671621e+08,3.989659e+07,8.240679e+06,6.497081e+06,5.666638e+06,5.115091e+06,1.042921e+07,2.153229e+06,4.451606e+08
2014Q1,4.226183e+08,3.752915e+07,8.900541e+06,5.766768e+06,4.702164e+06,4.558190e+06,1.011601e+07,2.601428e+06,4.967926e+08


### Vintage $ DQ Rate View

In [9]:
YQ_list = Perf[Perf['YQ']>='2015Q1']['YQ'].unique()
x = Perf.loc[Perf.groupby('Unique_Order_ID')['Year_and_Month'].idxmin()]

In [32]:
def vintage_view(df, YQ, vintage):
    order_list = vintage[vintage['YQ'] == YQ]['Unique_Order_ID'].unique()
    df_YQ = df[df['Unique_Order_ID'].isin(order_list)]
    df_DQ = df_YQ[df_YQ.Default_Status.isin(['151+', 'Chargeoff'])]
    df_DQ = df_DQ.loc[df_DQ.groupby('Unique_Order_ID')[
        'Year_and_Month'].idxmin()]
    view = df_DQ.groupby('Year_and_Month')[
        'Unique_Order_ID'].size().reset_index()
    temp = pd.DataFrame({'month_order': np.arange(1, 31), 'Year_and_Month': np.array(
        pd.date_range(start=YQ.to_timestamp(), periods=30, freq='MS'))})
    view = temp.merge(view, how = 'left',on='Year_and_Month')
    view = view.fillna(0)
    view['count'] = view['Unique_Order_ID'].cumsum()
    view['DQ_Rate'] = view['count']/len(order_list)
#     view = (df_YQ[df_YQ.Default_Status.isin(['151+','Chargeoff'])].pivot_table(values = 'Unique_Order_ID',index = 'Year_and_Month',columns = 'Default_Status',aggfunc='count')/len(order_list)).rename_axis(None, axis=1).reset_index()
#     view['month_order'] = np.arange(1+round((view['Year_and_Month'].min()-df_YQ['Year_and_Month'].min())/np.timedelta64(1, 'M')), len(
#         view)+1+round((view['Year_and_Month'].min()-df_YQ['Year_and_Month'].min())/np.timedelta64(1, 'M')))
    view.rename(columns={'DQ_Rate': str(YQ)+'_Vintage'}, inplace=True)
    return view

In [9]:
import datetime


In [11]:
YQ_list[0].to_timestamp()

Timestamp('2015-01-01 00:00:00')

In [14]:
df_DQ['Year_and_Month'].min()

Timestamp('2015-04-01 00:00:00')

In [22]:
 pd.date_range(start=YQ_list[0].to_timestamp(), periods=30, freq='MS')

DatetimeIndex(['2015-01-01', '2015-02-01', '2015-03-01', '2015-04-01', '2015-05-01', '2015-06-01', '2015-07-01', '2015-08-01', '2015-09-01', '2015-10-01', '2015-11-01', '2015-12-01', '2016-01-01', '2016-02-01', '2016-03-01', '2016-04-01', '2016-05-01', '2016-06-01', '2016-07-01', '2016-08-01', '2016-09-01', '2016-10-01', '2016-11-01', '2016-12-01', '2017-01-01', '2017-02-01', '2017-03-01', '2017-04-01', '2017-05-01', '2017-06-01'], dtype='datetime64[ns]', freq='MS')

In [23]:
[pd.Period(month=i)for i in range(30)]

ValueError: If value is None, freq cannot be None

In [22]:
[df_DQ['Year_and_Month'].min()+pd.Period('month')for i in range(30)]

ValueError: If value is None, freq cannot be None

In [17]:
temp = pd.DataFrame({'month_order':np.arange(1,31), 'Year_and_Month':np.array(pd.date_range(start=YQ.to_timestamp(), periods=30, freq='M'))})

In [18]:
temp

,month_order,Year_and_Month
0,1,2015-01-31
1,2,2015-02-28
2,3,2015-03-31
3,4,2015-04-30
4,5,2015-05-31
5,6,2015-06-30
6,7,2015-07-31
7,8,2015-08-31
8,9,2015-09-30
9,10,2015-10-31


In [42]:
# Q1
# temp = pd.DataFrame(np.arange(1,31),columns = ['month_order'])
Q1_vintage_list = []
for i in range(int(len(YQ_list)/4)):
    print(YQ_list[i*4])
    view = vintage_view(Perf, YQ_list[i*4], x)
#     Q1_vintage_list.append(view)
    
    Q1_vintage_list.append(view[['month_order',str(YQ_list[i*4])+'_Vintage']])

2015Q1
2016Q1
2017Q1
2018Q1
2019Q1


In [43]:
# Q2
Q2_vintage_list = []
for i in range(int(len(YQ_list)/4)):
    print(YQ_list[1+i*4])
    view = vintage_view(Perf, YQ_list[1+i*4], x)
    Q2_vintage_list.append(view[['month_order',str(YQ_list[1+i*4])+'_Vintage']])

2015Q2
2016Q2


KeyboardInterrupt: 

In [ ]:
# Q3
Q3_vintage_list = []
for i in range(int(len(YQ_list)/4)):
    print(YQ_list[2+i*4])
    view = vintage_view(Perf, YQ_list[2+i*4], x)
    Q3_vintage_list.append(view[['month_order',str(YQ_list[2+i*4])+'_Vintage']])

In [ ]:
# Q4
Q4_vintage_list = []
for i in range(int(len(YQ_list)/4)):
    print(YQ_list[3+i*4])
    view = vintage_view(Perf, YQ_list[3+i*4], x)
    Q4_vintage_list.append(view[['month_order',str(YQ_list[3+i*4])+'_Vintage']])

In [ ]:
Q1_final=reduce(lambda x,y: pd.merge(x,y, how='left',on='month_order'),Q1_vintage_list)
Q2_final=reduce(lambda x,y: pd.merge(x,y, how='left',on='month_order'),Q2_vintage_list)
Q3_final=reduce(lambda x,y: pd.merge(x,y, how='left',on='month_order'),Q3_vintage_list)
Q4_final=reduce(lambda x,y: pd.merge(x,y, how='left',on='month_order'),Q4_vintage_list)

In [ ]:
Q1_final.to_csv('../data/Q1.csv',index = False)
Q2_final.to_csv('../data/Q2.csv',index = False)
Q3_final.to_csv('../data/Q3.csv',index = False)
Q4_final.to_csv('../data/Q4.csv',index = False)

In [ ]:
def vintage_dollar_view(df, YQ, vintage):
    order_list = vintage[vintage['YQ'] == YQ]['Unique_Order_ID'].unique()
    total_balance = vintage[vintage['YQ'] == YQ]['Current_Balance'].sum()
    df_YQ = df[df['Unique_Order_ID'].isin(order_list)]
    df_DQ = df_YQ[df_YQ.Default_Status.isin(['151+', 'Chargeoff'])]
    df_DQ = df_DQ.loc[df_DQ.groupby('Unique_Order_ID')[
        'Year_and_Month'].idxmin()]
    view = df_DQ.groupby('Year_and_Month').agg({'Current_Balance':'sum'}).reset_index()
    temp = pd.DataFrame({'month_order': np.arange(1, 31), 'Year_and_Month': np.array(
        pd.date_range(start=YQ.to_timestamp(), periods=30, freq='MS'))})
    view = temp.merge(view, how = 'left',on='Year_and_Month')
    view = view.fillna(0)
    
    view['DQ_Balance'] = view['Current_Balance'].cumsum()
    view['DQ_Rate'] = view['DQ_Balance']/total_balance

#     view['month_order'] = np.arange(1+round((view['Year_and_Month'].min()-df_YQ['Year_and_Month'].min())/np.timedelta64(1, 'M')), len(
#         view)+1+round((view['Year_and_Month'].min()-df_YQ['Year_and_Month'].min())/np.timedelta64(1, 'M')))
    view.rename(columns = {'DQ_Rate':str(YQ)+'_Vintage'},inplace = True)
    return view

In [11]:
# Q1
# temp = pd.DataFrame(np.arange(1,31),columns = ['month_order'])
Q1_vintage_list = []
for i in range(int(len(YQ_list)/4)):
    print(YQ_list[i*4])
    view = vintage_dollar_view(Perf, YQ_list[i*4], x)
    Q1_vintage_list.append(view[['month_order',str(YQ_list[i*4])+'_Vintage']])

2015Q1
2016Q1
2017Q1
2018Q1
2019Q1


In [12]:
# Q2
Q2_vintage_list = []
for i in range(int(len(YQ_list)/4)):
    print(YQ_list[1+i*4])
    view = vintage_dollar_view(Perf, YQ_list[1+i*4], x)
    Q2_vintage_list.append(view[['month_order',str(YQ_list[1+i*4])+'_Vintage']])

2015Q2
2016Q2
2017Q2
2018Q2
2019Q2


In [14]:
# Q3
Q3_vintage_list = []
for i in range(int(len(YQ_list)/4)):
    print(YQ_list[2+i*4])
    view = vintage_dollar_view(Perf, YQ_list[2+i*4], x)
    Q3_vintage_list.append(view[['month_order',str(YQ_list[2+i*4])+'_Vintage']])

2015Q3
2016Q3
2017Q3
2018Q3
2019Q3


In [15]:
# Q4
Q4_vintage_list = []
for i in range(int(len(YQ_list)/4)-1):
    print(YQ_list[3+i*4])
    view = vintage_dollar_view(Perf, YQ_list[3+i*4], x)
    Q4_vintage_list.append(view[['month_order',str(YQ_list[3+i*4])+'_Vintage']])

2015Q4
2016Q4
2017Q4
2018Q4


In [16]:
Q1_final=reduce(lambda x,y: pd.merge(x,y, how='left',on='month_order'),Q1_vintage_list)
Q2_final=reduce(lambda x,y: pd.merge(x,y, how='left',on='month_order'),Q2_vintage_list)
Q3_final=reduce(lambda x,y: pd.merge(x,y, how='left',on='month_order'),Q3_vintage_list)
Q4_final=reduce(lambda x,y: pd.merge(x,y, how='left',on='month_order'),Q4_vintage_list)

In [21]:
Q1_final.to_csv('../data/Q1_Dollar.csv',index = False)
Q2_final.to_csv('../data/Q2_Dollar.csv',index = False)
Q3_final.to_csv('../data/Q3_Dollar.csv',index = False)
Q4_final.to_csv('../data/Q4_Dollar.csv',index = False)

In [20]:
Q4_final



,month_order,2015Q4_Vintage,2016Q4_Vintage,2017Q4_Vintage,2018Q4_Vintage
0,1,0.000000,0.000022,0.000000,0.000000
1,2,0.000000,0.000022,0.000000,0.000000
2,3,0.000000,0.000048,0.000006,0.000000
3,4,0.000000,0.000051,0.000006,0.000000
4,5,0.000000,0.000051,0.000006,0.000000
5,6,0.000097,0.000085,0.000030,0.000100
6,7,0.000930,0.001074,0.000952,0.001400
7,8,0.003961,0.005102,0.004987,0.006979
8,9,0.009272,0.011981,0.013462,0.017641
9,10,0.017346,0.023145,0.025143,0.032828


### Portfolio View - First Time DQ151+ Rate

In [23]:
DQ151 = Perf[Perf.Default_Status =='151+']
DQ151 = DQ151.loc[DQ151.groupby('Unique_Order_ID')['Year_and_Month'].idxmin()]
view = DQ151.groupby('YQ')['Unique_Order_ID'].size().reset_index()
view

,YQ,Unique_Order_ID
0,2012Q2,3
1,2012Q3,308
2,2012Q4,1166
3,2013Q1,2377
4,2013Q2,3083
5,2013Q3,5582
6,2013Q4,6958
7,2014Q1,6591
8,2014Q2,5331
9,2014Q3,7007


In [10]:
Perf['Default_Status'].unique()

array(['Current', '001-030', '061-090', '031-060', '121-150', '091-120',
       '151+', 'Chargeoff', 'Fraud', 'Bankruptcy'], dtype=object)

In [12]:
Perf.pivot_table(values='Current_Balance', index='Year_and_Month',
                 columns='Default_Status', aggfunc='sum', margins=True).to_csv('../data/DQ_bal_ds.csv')

In [13]:
Perf.pivot_table(values='Current_Balance', index='Year_and_Month',
                 columns='Delinquency_Bucket', aggfunc='sum', margins=True).to_csv('../data/DQ_bal.csv')

In [9]:
Perf[Perf['Default_Status'] == 'Chargeoff'].groupby(
    'Year_and_Month').agg({'Losses': 'sum'}).to_csv('../data/CO_Loss.csv')

,YQ,Unique_Order_ID
0,2012Q1,50256
1,2012Q2,143959
2,2012Q3,256584
3,2012Q4,431733
4,2013Q1,605728
5,2013Q2,655469
6,2013Q3,694354
7,2013Q4,764010
8,2014Q1,788795
9,2014Q2,788875


In [13]:
# Perf.pivot_table(values = 'Unique_Order_ID',index = 'YQ',columns='Delinquency_Bucket',aggfunc='count',margins = True)

### Why there is seasonal effect in DQ rate

In [ ]:
Perf.groupby('YQ')['Unique_Order_ID'].size().reset_index()

In [11]:
# New Order volume by Y-Q
Perf[Perf['Payment_Order'] == 1].pivot_table(
    values='Unique_Order_ID', index='YQ', columns='Delinquency_Bucket', aggfunc='count', margins=True)

Delinquency_Bucket,000-000,001-030,031-060,061-090,091-120,121-150,151+,All
YQ,,,,,,,,
2012Q1,26405.0,80.0,NaN,NaN,NaN,NaN,NaN,26485
2012Q2,31276.0,161.0,2.0,NaN,NaN,NaN,NaN,31439
2012Q3,40538.0,283.0,5.0,1.0,NaN,1.0,NaN,40828
2012Q4,82503.0,451.0,6.0,1.0,NaN,NaN,NaN,82961
2013Q1,39606.0,335.0,6.0,1.0,NaN,NaN,NaN,39948
2013Q2,42241.0,315.0,12.0,10.0,2.0,3.0,12.0,42595
2013Q3,45598.0,233.0,1.0,NaN,NaN,NaN,NaN,45832
2013Q4,99155.0,654.0,7.0,2.0,1.0,1.0,13.0,99833
2014Q1,42229.0,162.0,4.0,NaN,NaN,NaN,1.0,42396


In [14]:
list(Perf)

['Unique_Order_ID',
 'Unique_Customer_ID',
 'Order_Amount',
 'Product_Markup',
 'Payments_Due',
 'Current_Balance',
 'Date_account_went_delinquent',
 'Default_Status',
 'Delinquency_Bucket',
 'Losses',
 'Auto_Pay_Enrollment',
 'Order_Closed_Flag',
 'Type_of_Backup_Payment',
 'Spending_Limit_at_time_of_Order',
 'Tenure_at_time_of_Order',
 'Salary_at_time_of_Order',
 'Street1_x',
 'Street2_x',
 'City_x',
 'State_x',
 'Zip_x',
 'items_ordered',
 'Year_and_Month',
 'EMPSTATUS',
 'Order_Date',
 'Unique_Company_ID',
 'Enrollment_Date',
 'Assigned_Spending_Limit',
 'Customer_Tenure',
 'Customer_Salary',
 'Prior_Customer_Flag',
 'Street1_y',
 'Street2_y',
 'City_y',
 'State_y',
 'Zip_y',
 'Status',
 'Term_Date',
 'Active',
 'Client ID',
 'Account Name',
 'Industry',
 'NAICS Code',
 'NAICS Description',
 'Broker',
 'TPA_x',
 'Launch Date',
 'Termination Date',
 'Affiliate/Fed Gov?',
 'Cash',
 'YQ',
 'Payment_Order']

In [16]:
Perf[Perf['Payment_Order'] == 1].pivot_table(values = 'Unique_Order_ID',index = 'YQ',columns = 'Order_Date',aggfunc = 'count')

Order_Date,01/01/2012,01/01/2013,01/01/2014,01/01/2015,01/01/2016,01/01/2017,01/01/2018,01/01/2019,01/02/2012,01/02/2013,01/02/2014,01/02/2015,01/02/2016,01/02/2017,01/02/2018,01/02/2019,01/03/2012,01/03/2013,01/03/2014,01/03/2015,01/03/2016,01/03/2017,01/03/2018,01/03/2019,01/04/2012,01/04/2013,01/04/2014,01/04/2015,01/04/2016,01/04/2017,01/04/2018,01/04/2019,01/05/2012,01/05/2013,01/05/2014,01/05/2015,01/05/2016,01/05/2017,01/05/2018,01/05/2019,01/06/2012,01/06/2013,01/06/2014,01/06/2015,01/06/2016,01/06/2017,01/06/2018,01/06/2019,01/07/2012,01/07/2013,01/07/2014,01/07/2015,01/07/2016,01/07/2017,01/07/2018,01/07/2019,01/08/2012,01/08/2013,01/08/2014,01/08/2015,01/08/2016,01/08/2017,01/08/2018,01/08/2019,01/09/2012,01/09/2013,01/09/2014,01/09/2015,01/09/2016,01/09/2017,01/09/2018,01/09/2019,01/10/2012,01/10/2013,01/10/2014,01/10/2015,01/10/2016,01/10/2017,01/10/2018,01/10/2019,01/11/2012,01/11/2013,01/11/2014,01/11/2015,01/11/2016,01/11/2017,01/11/2018,01/11/2019,01/12/2012,01/12/2013,01/12/2014,01/12/2015,01/12/2016,01/12/2017,01/12/2018,01/12/2019,01/13/2012,01/13/2013,01/13/2014,01/13/2015,01/13/2016,01/13/2017,01/13/2018,01/13/2019,01/14/2012,01/14/2013,01/14/2014,01/14/2015,01/14/2016,01/14/2017,01/14/2018,01/14/2019,01/15/2012,01/15/2013,01/15/2014,01/15/2015,01/15/2016,01/15/2017,01/15/2018,01/15/2019,01/16/2012,01/16/2013,01/16/2014,01/16/2015,01/16/2016,01/16/2017,01/16/2018,01/16/2019,01/17/2012,01/17/2013,01/17/2014,01/17/2015,01/17/2016,01/17/2017,01/17/2018,01/17/2019,01/18/2012,01/18/2013,01/18/2014,01/18/2015,01/18/2016,01/18/2017,01/18/2018,01/18/2019,01/19/2012,01/19/2013,01/19/2014,01/19/2015,01/19/2016,01/19/2017,01/19/2018,01/19/2019,01/20/2012,01/20/2013,01/20/2014,01/20/2015,01/20/2016,01/20/2017,01/20/2018,01/20/2019,01/21/2012,01/21/2013,01/21/2014,01/21/2015,01/21/2016,01/21/2017,01/21/2018,01/21/2019,01/22/2012,01/22/2013,01/22/2014,01/22/2015,01/22/2016,01/22/2017,01/22/2018,01/22/2019,01/23/2012,01/23/2013,01/23/2014,01/23/2015,01/23/2016,01/23/2017,01/23/2018,01/23/2019,01/24/2012,01/24/2013,01/24/2014,01/24/2015,01/24/2016,01/24/2017,01/24/2018,01/24/2019,01/25/2012,01/25/2013,01/25/2014,01/25/2015,01/25/2016,01/25/2017,01/25/2018,01/25/2019,01/26/2012,01/26/2013,01/26/2014,01/26/2015,01/26/2016,01/26/2017,01/26/2018,01/26/2019,01/27/2012,01/27/2013,01/27/2014,01/27/2015,01/27/2016,01/27/2017,01/27/2018,01/27/2019,01/28/2012,01/28/2013,01/28/2014,01/28/2015,01/28/2016,01/28/2017,01/28/2018,01/28/2019,01/29/2012,01/29/2013,01/29/2014,01/29/2015,01/29/2016,01/29/2017,01/29/2018,01/29/2019,01/30/2012,01/30/2013,01/30/2014,01/30/2015,01/30/2016,01/30/2017,01/30/2018,01/30/2019,01/31/2012,01/31/2013,01/31/2014,01/31/2015,01/31/2016,01/31/2017,01/31/2018,01/31/2019,02/01/2012,02/01/2013,...,11/26/2014,11/26/2015,11/26/2016,11/26/2017,11/26/2018,11/27/2012,11/27/2013,11/27/2014,11/27/2015,11/27/2016,11/27/2017,11/27/2018,11/28/2012,11/28/2013,11/28/2014,11/28/2015,11/28/2016,11/28/2017,11/28/2018,11/29/2012,11/29/2013,11/29/2014,11/29/2015,11/29/2016,11/29/2017,11/29/2018,11/30/2012,11/30/2013,11/30/2014,11/30/2015,11/30/2016,11/30/2017,11/30/2018,12/01/2012,12/01/2013,12/01/2014,12/01/2015,12/01/2016,12/01/2017,12/01/2018,12/02/2012,12/02/2013,12/02/2014,12/02/2015,12/02/2016,12/02/2017,12/02/2018,12/03/2012,12/03/2013,12/03/2014,12/03/2015,12/03/2016,12/03/2017,12/03/2018,12/04/2012,12/04/2013,12/04/2014,12/04/2015,12/04/2016,12/04/2017,12/04/2018,12/05/2012,12/05/2013,12/05/2014,12/05/2015,12/05/2016,12/05/2017,12/05/2018,12/06/2012,12/06/2013,12/06/2014,12/06/2015,12/06/2016,12/06/2017,12/06/2018,12/07/2012,12/07/2013,12/07/2014,12/07/2015,12/07/2016,12/07/2017,12/07/2018,12/08/2012,12/08/2013,12/08/2014,12/08/2015,12/08/2016,12/08/2017,12/08/2018,12/09/2012,12/09/2013,12/09/2014,12/09/2015,12/09/2016,12/09/2017,12/09/2018,12/10/2012,12/10/2013,12/10/2014,12/10/2015,12/10/2016,12/10/2017,12/10/2018,12/11/2012,12/11/2013,12/11/2014,12/11/2015,12/11/2016,12/11/2017,12/11/2018,12/12/2012,12/12/2013,12/

In [18]:
pd.Period('2012Q1')

Period('2012Q1', 'Q-DEC')

In [22]:
Perf[(Perf['Payment_Order'] == 1) & (Perf['YQ'] >= pd.Period('2015Q1'))].groupby(['YQ','Order_Date']).agg({'Unique_Order_ID':'count'}).reset_index()

,YQ,Order_Date,Unique_Order_ID
0,2015Q1,01/01/2015,406
1,2015Q1,01/02/2015,769
2,2015Q1,01/03/2015,442
3,2015Q1,01/04/2015,367
4,2015Q1,01/05/2015,577
5,2015Q1,01/06/2015,637
6,2015Q1,01/07/2015,639
7,2015Q1,01/08/2015,656
8,2015Q1,01/09/2015,595
9,2015Q1,01/10/2015,467
